In [2]:
import imaplib
import email
from email.header import decode_header
from getpass import getpass
# Dane do logowania

In [ ]:
login = 'rkorzen@wp.pl'
haslo = getpass()

In [ ]:

def odczytaj_cialo_wiadomosci(e_mail):
    if e_mail.is_multipart():
        # Dla przypadku, gdy wiadomość jest multipart (ma części)
        for part in e_mail.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))
            
            # Szukamy tylko części, które nie mają dodatkowych "Content-Disposition"
            if content_type == "text/plain" and "attachment" not in content_disposition:
                # Dekodujemy ciało wiadomości
                text = part.get_payload(decode=True).decode()
                return text
            elif content_type == "text/html" and "attachment" not in content_disposition:
                html = part.get_payload(decode=True).decode()
                return html
    else:
        # Dla przypadku, gdy wiadomość nie jest multipart
        content_type = e_mail.get_content_type()
        if content_type == "text/plain":
            text = e_mail.get_payload(decode=True).decode()
            return text
        if content_type == "text/html":
            html = e_mail.get_payload(decode=True).decode()
            return html
    return ""

In [ ]:
# Łączenie się z serwerem IMAP
with imaplib.IMAP4_SSL('imap.wp.pl') as mail:
    mail.login(login, haslo)
    
    # Wybór skrzynki odbiorczej
    mail.select('inbox')
    
    # Szukanie wiadomości - tutaj wszystkie
    typ, dane_wiadomosci = mail.search(None, 'ALL')
    
    for numer in dane_wiadomosci[0].split():
        typ, dane = mail.fetch(numer, '(RFC822)')
        wiadomosc = email.message_from_bytes(dane[0][1])
        
        # Dekodowanie tematu wiadomości
        temat = decode_header(wiadomosc['Subject'])[0][0]
        try:
            if isinstance(temat, bytes):
                temat = temat.decode()
            print('Temat:', temat)
        except:
            pass
        # Tutaj możesz dodać więcej logiki do przetwarzania wiadomości,
        # np. odczytywanie ciała wiadomości, pobieranie załączników itp.
        
        if temat == "test":
            
            print(odczytaj_cialo_wiadomosci(wiadomosc))
            break
    mail.logout()